<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/cover-small.jpg">

*Este libro es una versión al español de [Python for Everybody](https://www.py4e.com/) escrito por el [Dr. Charles R. Severance](http://www.dr-chuck.com/); este contenido esta disponible en [GitHub](https://github.com/csev/py4e).*

Detalles de Copyright

*Copyright ~ 2009- Charles Severance.
Este trabajo está registrado bajo una Licencia Creative Commons AttributionNonCommercial-ShareAlike 3.0 [CC BY-NC-SA](https://creativecommons.org/licenses/by-nc-sa/3.0/).*

<!--NAVIGATION-->
| [Indice](indice.ipynb) | 

< [Capítulo 14 - Programación orientada a objetos](cap14.ipynb) | [Capítulo 16 - Visualización de datos](cap16.ipynb) >

# Capítulo 15 - Uso de bases de datos y SQL

## ¿Que es una base de datos?

Una base de datos es un archivo que está organizado para almacenar datos. La mayoría de las bases de datos están organizadas como un diccionario en el sentido de que se asignan desde claves a valores. La mayor diferencia es que  la base de datos está en el disco (u otro almacenamiento permanente), por lo que persiste una vez que finaliza el programa. Debido a que una base de datos se almacena en almacenamiento permanente, puede almacenar muchos más datos que un diccionario, que está limitado al tamaño de la memoria en la computadora.

Al igual que un diccionario, el software de base de datos está diseñado para mantener la inserción y el acceso a los datos muy rápido, incluso para grandes cantidades de datos. El software de base de datos mantiene su rendimiento mediante la creación de índices a medida que se agregan datos a la base de datos para permitir que la computadora salte rápidamente a una entrada en particular.

Hay muchos sistemas de bases de datos diferentes que se utilizan para una amplia variedad de propósitos, incluidos: Oracle, MySQL, Microsoft SQL Server, PostgreSQL y SQLite. Nos centramos en SQLite en este libro porque es una base de datos muy común y ya está integrada en Python. SQLite está diseñado para integrarse en otras aplicaciones para proporcionar soporte de base de datos dentro de la aplicación. Por ejemplo, el navegador Firefox también usa la base de datos SQLite internamente como lo hacen muchos otros productos http://sqlite.org/.

SQLite se adapta bien a algunos de los problemas de manipulación de datos que vemos en Informática, como la aplicación de rastreo de Twitter que describimos en este capítulo.

## Conceptos de base de datos

Cuando miras por primera vez una base de datos, se ve como una hoja de cálculo con varias hojas. Las estructuras de datos principales en una base de datos son: tablas, filas y columnas.

![Figura 15.1](./figures/15.1.svg)
<center><i>Figura 15.1: Bases de datos relacionales.</i></center>

En las descripciones técnicas de las bases de datos relacionales, los conceptos de tabla, fila y columna se conocen más formalmente como relación, tupla y atributo, respectivamente. Usaremos los términos menos formales en este capítulo.

## Navegador de base de datos para SQLite

Si bien este capítulo se centrará en el uso de Python para trabajar con datos en archivos de base de datos SQLite, se pueden realizar muchas operaciones de manera más conveniente mediante el software denominado Navegador de bases de datos para SQLite, que está disponible de forma gratuita desde http://sqlitebrowser.org/.

Con el navegador puede crear fácilmente tablas, insertar datos, editar datos o ejecutar consultas SQL simples sobre los datos en la base de datos.

En cierto sentido, el navegador de la base de datos es similar a un editor de texto cuando se trabaja con archivos de texto. Cuando desee realizar una o muy pocas operaciones en un archivo de texto, puede simplemente abrirlo en un editor de texto y realizar los cambios que desee. Cuando tenga que hacer muchos cambios en un archivo de texto, a menudo escribirá un programa simple de Python. Encontrará el mismo patrón cuando trabaje con bases de datos. Harás operaciones simples en el administrador de bases de datos y las operaciones más complejas se realizarán de forma más conveniente en Python.

## Creando una tabla de base de datos

Las bases de datos requieren una estructura más definida que las listas o los diccionarios de Python.

Cuando creamos una tabla de base de datos , debemos informar a la base de datos con anticipación los nombres de cada una de las columnas en la tabla y el tipo de datos que planeamos almacenar en cada columna . Cuando el software de la base de datos conoce el tipo de datos en cada columna, puede elegir la manera más eficiente de almacenar y buscar los datos según el tipo de datos.

Puede ver los diversos tipos de datos soportados por SQLite en la siguiente url: http://www.sqlite.org/datatypes.html.

Definir la estructura de sus datos por adelantado puede parecer incómodo al principio, pero la recompensa es un acceso rápido a sus datos, incluso cuando la base de datos contiene una gran cantidad de datos.

El código para crear un archivo de base de datos y una tabla nombrada Tracks con dos columnas en la base de datos es el siguiente:

In [1]:
import sqlite3

conn = sqlite3.connect('music.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Tracks')
cur.execute('CREATE TABLE Tracks (title TEXT, plays INTEGER)')

conn.close()

La operación `connect` realiza una "conexión" a la base de datos almacenada en el archivo `music.sqlite3` en el directorio actual. Si el archivo no existe, se creará. La razón por la que esto se llama "conexión" es que a veces la base de datos se almacena en un "servidor de base de datos" separado del servidor en el que estamos ejecutando nuestra aplicación. En nuestros ejemplos simples, la base de datos solo será un archivo local en el mismo directorio que el código de Python que estamos ejecutando.

Un cursor es como un identificador de archivo que podemos usar para realizar operaciones en los datos almacenados en la base de datos. Llamar `cursor()` es muy similar conceptualmente a las llamadas `open()` cuando se trata de archivos de texto.

![Figura 15.2](./figures/15.2.svg)
<center><i>Figura 15.2: Un cursor de base de datos</i></center>

Una vez que tenemos el cursor, podemos comenzar a ejecutar comandos sobre los contenidos de la base de datos usando el método `execute()`.

Los comandos de la base de datos se expresan en un lenguaje especial que se ha estandarizado en muchos
proveedores de bases de datos diferentes para permitirnos aprender un único lenguaje de base de datos. El lenguaje de la base de datos se llama lenguaje de consulta estructurado o SQL para abreviar
http://en.wikipedia.org/wiki/SQL. 

En nuestro ejemplo, estamos ejecutando dos comandos SQL en nuestra base de datos. Como una convención,
mostraremos las palabras clave SQL en mayúsculas y las partes del comando que estamos agregando (como los nombres de las tablas y columnas) se mostrarán en minúsculas.

El primer comando SQL elimina la tabla Tracks de la base de datos si existe. Este patrón es simplemente para permitirnos ejecutar el mismo programa para crear la tabla Tracks una y otra vez sin causar un error. Tenga en cuenta que el comando `DROP TABLE` elimina la tabla y todos sus contenidos de la base de datos (es decir, no hay "deshacer").

    cur.execute('DROP TABLE IF EXISTS Tracks ')

El segundo comando crea una tabla nombrada Tracks con una columna de texto nombrada `title` y una columna entera nombrada plays.

    cur.execute('CREATE TABLE Tracks (title TEXT, plays INTEGER)')

Ahora que hemos creado una tabla con el nombre Tracks, podemos poner algunos datos en esa tabla usando la operación SQL `INSERT`. De nuevo, comenzamos haciendo una conexión a la base de datos y obteniendo el cursor. Entonces podemos ejecutar comandos SQL usando el cursor.

El comando SQL `INSERT` indica qué tabla estamos usando y luego define una nueva fila al enumerar los campos que queremos incluir (title, plays) seguidos de los `VALUES` que queremos colocar en la nueva fila. Especificamos los valores como signos de interrogación `(?, ?)` para indicar que los valores reales se pasan como una tupla `('My Way', 15)` como el segundo parámetro de la llamada `execute()`.

In [2]:
import sqlite3

conn = sqlite3.connect('music.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Tracks')
cur.execute('CREATE TABLE Tracks (title TEXT, plays INTEGER)')

cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)', ('Thunderstruck', 20))
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)',  ('My Way', 15))
conn.commit()

print('Tracks:')
cur.execute('SELECT title, plays FROM Tracks')
for row in cur:
    print(row)

cur.execute('DELETE FROM Tracks WHERE plays < 100')

cur.close()

Tracks:
('Thunderstruck', 20)
('My Way', 15)


Primero, `INSERT` inserta dos filas en nuestra tabla y utilizamos `commit()` para forzar que los datos se escriban en el archivo de la base de datos.

![Figura 15.3](./figures/15.3.svg)
<center><i>Figura 15.3: Filas en una tabla</i></center>

Luego usamos el comando `SELECT` para recuperar las filas que acabamos de insertar de la tabla. En el comando `SELECT`, indicamos qué columnas queremos `(title, plays)` e indicamos de qué tabla queremos recuperar los datos. Después de que ejecutamos la instrucción `SELECT`, el cursor es algo que podemos recorrer en una instrucción `for`. Para mayor eficiencia, el cursor no lee todos los datos de la base de datos cuando ejecutamos la declaración `SELECT`. En cambio, los datos se leen a medida que recorremos las filas en la declaración `for`.

Nuestro bucle `for` encuentra dos filas, y cada fila es una tupla de Python con el primer valor como `title` y el segundo valor como el número de reproducciones `plays`.

Nota: Puede ver cadenas que comienzan `u'` en otros libros o en Internet. Esto fue una indicación en Python 2 de que las cadenas son cadenas Unicode *que son capaces de almacenar conjuntos de caracteres no latinos. En Python 3, todas las cadenas son cadenas unicode por defecto*.

Al final del programa, ejecutamos un comando SQL `DELETE` en las filas que acabamos de crear para que podamos ejecutar el programa una y otra vez. El comando `DELETE` muestra el uso de una cláusula `WHERE` que nos permite expresar un criterio de selección para que podamos solicitar a la base de datos que aplique el comando solo a las filas que coinciden con el criterio. En este ejemplo, el criterio se aplica a todas las filas, por lo que vaciamos la tabla para que podamos ejecutar el programa repetidamente. Después de que `DELETE` se realiza, también llamamos `commit()` para forzar que los datos se eliminen de la base de datos.

## Resumen del lenguaje de consulta estructurado

Hasta ahora, hemos utilizado el lenguaje de consulta estructurado en nuestros ejemplos de Python y hemos cubierto muchos de los conceptos básicos de los comandos de SQL. En esta sección, observamos el lenguaje SQL en particular y ofrecemos una descripción general de la sintaxis SQL.

Debido a que hay tantos proveedores de bases de datos diferentes, el Lenguaje de consulta estructurado (SQL) se estandarizó para que pudiéramos comunicarnos de manera portátil a los sistemas de bases de datos de múltiples proveedores.

Una base de datos relacional se compone de tablas, filas y columnas. Las columnas generalmente tienen un tipo como texto, numérico o datos de fecha. Cuando creamos una tabla, indicamos los nombres y tipos de las columnas:

    CREATE TABLE Tracks (title TEXT, plays INTEGER)

Para insertar una fila en una tabla, usamos el comando SQL `INSERT`:

    INSERT INTO Tracks (title, plays) VALUES ('My Way', 15)
    
La instrucción `INSERT` especifica el nombre de la tabla, luego una lista de los campos/columnas que le gustaría establecer en la nueva fila, y luego la palabra clave `VALUES` y una lista de valores correspondientes para cada uno de los campos.

El comando SQL `SELECT` se usa para recuperar filas y columnas de una base de datos. La declaración `SELECT` le permite especificar qué columnas le gustaría recuperar, así como una cláusula `WHERE` para seleccionar qué filas le gustaría ver. También permite una cláusula opcional llamada `ORDER BY` para controlar la clasificación de las filas devueltas.

    SELECT * FROM Tracks WHERE title = 'My Way'

El uso de * indica que desea que la base de datos devuelva todas las columnas para cada fila que coincida con la cláusula `WHERE`.

Tenga en cuenta que, a diferencia de Python, en una cláusula `WHERE` de SQL utilizamos un solo signo igual para indicar una prueba de igualdad en lugar de un signo de doble igual. Otras operaciones lógicas permitidas en una cláusula `WHERE` incluyen `<`, `>`, `<=`, `>=`, `!=`,, así como `AND` y `OR` y los paréntesis para construir sus expresiones lógicas.

Puede solicitar que las filas devueltas se ordenen por uno de los campos de la siguiente manera:

    SELECT title,plays FROM Tracks ORDER BY title

Para eliminar una fila, necesita una cláusula `WHERE` en una declaración SQL `DELETE`. La cláusula `WHERE` determina qué filas se van a eliminar:

    DELETE FROM Tracks WHERE title = 'My Way'

Es posible actualizar con `UPDATE` una columna o columnas dentro de una o más filas en una tabla utilizando la declaración SQL `UPDATE` de la siguiente manera:

    UPDATE Tracks SET plays = 16 WHERE title = 'My Way'

La declaración `UPDATE` especifica una tabla y luego una lista de campos y valores para cambiar después de la palabra clave `SET` y luego una cláusula `WHERE` opcional para seleccionar las filas que se van a actualizar. Una sola instrucción `UPDATE` cambiará todas las filas que coinciden con la cláusula `WHERE`. Si `WHERE` no se especifica una cláusula, realiza el `UPDATE` en todas las filas de la tabla.

Estos cuatro comandos SQL básicos `INSERT`, `SELECT`, `UPDATE` y `DELETE` permiten las cuatro operaciones básicas necesarias para crear y mantener datos.

## Spidering Twitter usando una base de datos

En esta sección, crearemos un programa de spidering simple que pasará por las cuentas de Twitter y construirá una base de datos de ellas. Nota: Tenga mucho cuidado al ejecutar este programa. No desea extraer demasiados datos o ejecutar el programa durante demasiado tiempo y terminar teniendo su acceso de Twitter denegado.

Uno de los problemas de cualquier tipo de programa de spidering es que necesita ser detenido y reiniciado muchas veces y no quiere perder los datos que ha recuperado hasta ahora. No siempre debe reiniciar su recuperación de datos desde el principio, por lo que queremos almacenar los datos a medida que los recuperamos para que nuestro programa pueda iniciar una copia de seguridad y continuar donde lo dejó.

Comenzaremos por recuperar los amigos de Twitter de una persona y sus estados, recorriendo la lista de amigos y agregando cada uno de los amigos a una base de datos para recuperar en el futuro. Después de procesar a los amigos de Twitter de una persona, verificamos en nuestra base de datos y recuperamos a uno de los amigos del amigo. Hacemos esto una y otra vez, escogiendo a una persona "no visitada", recuperando su lista de amigos y agregando amigos que no hemos visto en nuestra lista para una futura visita.

También hacemos un seguimiento de cuántas veces hemos visto a un amigo en particular en la base de datos para tener una idea de su "popularidad".

Al almacenar nuestra lista de cuentas conocidas y si hemos recuperado la cuenta o no, y qué tan popular es la cuenta en una base de datos en el disco de la computadora, podemos detener y reiniciar nuestro programa tantas veces como lo deseemos.

Este programa es un poco complejo. Se basa en el código del ejercicio anterior del libro que usa la API de Twitter.

Aquí está el código fuente de nuestra aplicación de Twitter spidering:

    from urllib.request import urlopen
    import urllib.error
    import twurl
    import json
    import sqlite3
    import ssl

    TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'

    conn = sqlite3.connect('spider.sqlite')
    cur = conn.cursor()

    cur.execute('''
                CREATE TABLE IF NOT EXISTS Twitter
                (name TEXT, retrieved INTEGER, friends INTEGER)''')

    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    while True:
        acct = input('Enter a Twitter account, or quit: ')
        if (acct == 'quit'): break
        if (len(acct) < 1):
            cur.execute('SELECT name FROM Twitter WHERE retrieved = 0 LIMIT 1')
            try:
                acct = cur.fetchone()[0]
            except:
                print('No unretrieved Twitter accounts found')
                continue

        url = twurl.augment(TWITTER_URL, {'screen_name': acct, 'count': '5'})
        print('Retrieving', url)
        connection = urlopen(url, context=ctx)
        data = connection.read().decode()
        headers = dict(connection.getheaders())

        print('Remaining', headers['x-rate-limit-remaining'])
        js = json.loads(data)
        # Debugging
        # print json.dumps(js, indent=4)

        cur.execute('UPDATE Twitter SET retrieved=1 WHERE name = ?', (acct, ))

        countnew = 0
        countold = 0
        for u in js['users']:
            friend = u['screen_name']
            print(friend)
            cur.execute('SELECT friends FROM Twitter WHERE name = ? LIMIT 1',
                        (friend, ))
            try:
                count = cur.fetchone()[0]
                cur.execute('UPDATE Twitter SET friends = ? WHERE name = ?',
                            (count+1, friend))
                countold = countold + 1
            except:
                cur.execute('''INSERT INTO Twitter (name, retrieved, friends)
                            VALUES (?, 0, 1)''', (friend, ))
                countnew = countnew + 1
        print('New accounts=', countnew, ' revisited=', countold)
        conn.commit()

    cur.close()

Nuestra base de datos se almacena en el archivo spider.sqlite3 y tiene una tabla nombrada Twitter. Cada fila en la tabla Twitter tiene una columna para el nombre de la cuenta, ya sea que hayamos recuperado los amigos de esta cuenta, y cuántas veces esta cuenta ha sido "amigable".

En el ciclo principal del programa, solicitamos al usuario un nombre de cuenta de Twitter o "salir" para salir del programa. Si el usuario ingresa una cuenta de Twitter, recuperamos la lista de amigos y estados para ese usuario y agregamos cada amigo a la base de datos, si aún no está en la base de datos. Si el amigo ya está en la lista, agregamos 1 al campo `friends` en la fila en la base de datos.

Si el usuario presiona enter, buscamos en la base de datos la siguiente cuenta de Twitter que aún no hayamos recuperado, recuperamos los amigos y los estados para esa cuenta, los agregamos a la base de datos o los actualizamos, y aumentamos su conteo de `friends`.

Una vez que recuperamos la lista de amigos y estados, recorremos todos los elementos `user` en el JSON devuelto y recuperamos el `screen_name` de cada usuario. Luego usamos la declaración `SELECT` para ver si ya hemos almacenado este `screen_name` particular en la base de datos y recuperamos el recuento de amigos `friends` si el registro existe.

    countnew = 0
    countold = 0
    for u in js['users'] :
        friend = u['screen_name']
        print(friend)
        cur.execute('SELECT friends FROM Twitter WHERE name = ? LIMIT 1',
            (friend, ) )
        try:
            count = cur.fetchone()[0]
            cur.execute('UPDATE Twitter SET friends = ? WHERE name = ?',
                (count+1, friend) )
            countold = countold + 1
        except:
            cur.execute('''INSERT INTO Twitter (name, retrieved, friends)
                VALUES ( ?, 0, 1 )''', ( friend, ) )
            countnew = countnew + 1
    print('New accounts=',countnew,' revisited=',countold)
    conn.commit()

Una vez que el cursor ejecuta la instrucción `SELECT`, debemos recuperar las filas. Podríamos hacer esto con una declaración `for`, pero como solo estamos recuperando una fila `LIMIT 1`, podemos usar el método `fetchone()` para obtener la primera (y única) fila que es el resultado de la operación `SELECT`. Como `fetchone()` devuelve la fila como una tupla (aunque solo haya un campo), tomamos el primer valor de la tupla usando para obtener el recuento actual de amigos en la variable `count`.

Si esta recuperación es exitosa, usamos la instrucción SQL `UPDATE` con una cláusula `WHERE` para agregar 1 a la columna `friends` de la fila que coincide con la cuenta del amigo. Observe que hay dos marcadores de posición (es decir, signos de interrogación) en el SQL, y el segundo parámetro para el `execute()` es una tupla de dos elementos que contiene los valores que se sustituirán en el SQL en lugar de los signos de interrogación.

Si el código en el bloque `try` falla, es probable que sea porque ningún registro coincide con la cláusula `WHERE name = ?` en la instrucción `SELECT`. Entonces, en el bloque `except`, usamos la declaración de SQL `INSERT` para agregar el amigo `screen_name` a la tabla con una indicación de que aún no hemos recuperado `screen_name` y establecemos el recuento de amigos en cero.

Entonces, la primera vez que se ejecuta el programa e ingresamos una cuenta de Twitter, el programa se ejecuta de la siguiente manera:

    Enter a Twitter account, or quit: drchuck
    Retrieving http://api.twitter.com/1.1/friends ...
    New accounts= 20  revisited= 0
    Enter a Twitter account, or quit: quit

Como esta es la primera vez que ejecutamos el programa, la base de datos está vacía y creamos la base de datos en el archivo `spider.sqlite3` y agregamos una tabla con el nombre `Twitter` de la base de datos. Luego recuperamos algunos amigos y los agregamos todos a la base de datos ya que la base de datos está vacía.

En este punto, es posible que deseemos escribir un volcador de base de datos simple para echar un vistazo a lo que está en nuestro archivo `spider.sqlite3`:

    import sqlite3

    conn = sqlite3.connect('spider.sqlite')
    cur = conn.cursor()
    cur.execute('SELECT * FROM Twitter')
    count = 0
    for row in cur:
        print(row)
        count = count + 1
    print(count, 'rows.')
    cur.close()

Este programa simplemente abre la base de datos y selecciona todas las columnas de todas las filas en la tabla Twitter, luego recorre las filas e imprime cada fila.

Si ejecutamos este programa después de la primera ejecución de nuestra araña de Twitter, su resultado será el siguiente:

    ('opencontent', 0, 1)
    ('lhawthorn', 0, 1)
    ('steve_coppin', 0, 1)
    ('davidkocher', 0, 1)
    ('hrheingold', 0, 1)
    ...
    20 rows.

Vemos una fila para cada `screen_name`, que no hemos recuperado los datos para eso `screen_name`, y todos en la base de datos tienen un amigo.

Ahora nuestra base de datos refleja la recuperación de los amigos de nuestra primera cuenta de Twitter (drchuck). Podemos ejecutar el programa nuevamente y decirle que recupere a los amigos de la siguiente cuenta "sin procesar" simplemente presionando Enter en lugar de una cuenta de Twitter de la siguiente manera:

    Enter a Twitter account, or quit:
    Retrieving http://api.twitter.com/1.1/friends ...
    New accounts= 18  revisited= 2
    Enter a Twitter account, or quit:
    Retrieving http://api.twitter.com/1.1/friends ...
    New accounts= 17  revisited= 3
    Enter a Twitter account, or quit: quit

Como presionamos enter (es decir, no especificamos una cuenta de Twitter), se ejecuta el siguiente código:

    if ( len(acct) < 1 ) :
        cur.execute('SELECT name FROM Twitter WHERE retrieved = 0 LIMIT 1')
        try:
            acct = cur.fetchone()[0]
        except:
            print('No unretrieved twitter accounts found')
            continue

Usamos la declaración de SQL `SELECT` para recuperar el nombre del primer usuario () `LIMIT 1` que todavía tiene su valor "hemos recuperado este usuario" establecido en cero. También usamos el patrón `fetchone()[0]` dentro de un bloque `try`/`except` para extraer `screen_name` a partir de los datos recuperados o emitir un mensaje de error y hacer una copia de seguridad.

Si recuperamos con éxito un sin procesar `screen_name`, recuperamos sus datos de la siguiente manera:

    url=twurl.augment(TWITTER_URL,{'screen_name': acct,'count': '20'})
    print('Retrieving', url)
    connection = urllib.urlopen(url)
    data = connection.read()
    js = json.loads(data)

    cur.execute('UPDATE Twitter SET retrieved=1 WHERE name = ?',(acct, ))

Una vez que recuperamos los datos con éxito, usamos la declaración `UPDATE` para establecer la columna `retrieved` en 1 para indicar que hemos completado la recuperación de los amigos de esta cuenta. Esto nos impide recuperar los mismos datos una y otra vez y nos mantiene avanzando a través de la red de amigos de Twitter.

Si ejecutamos el programa amigo y presionamos Enter dos veces para recuperar a los amigos del amigo no visitado, luego ejecutamos el programa de dumping, nos dará el siguiente resultado:

    ('opencontent', 1, 1)
    ('lhawthorn', 1, 1)
    ('steve_coppin', 0, 1)
    ('davidkocher', 0, 1)
    ('hrheingold', 0, 1)
    ...
    ('cnxorg', 0, 2)
    ('knoop', 0, 1)
    ('kthanos', 0, 2)
    ('LectureTools', 0, 1)
    ...
    55 rows.

Podemos ver que hemos registrado correctamente que hemos visitado `lhawthorn` y `opencontent`. También las cuentas `cnxorg` y `kthanos` ya tienen dos seguidores. Y ahora que hemos recuperado los amigos de tres personas (drchuck, opencontent y lhawthorn) nuestra tabla tiene 55 filas de amigos a recuperar.

Cada vez que ejecutamos el programa y presionamos Enter, elegiremos la siguiente cuenta no visitada (por ej., La próxima cuenta será steve_coppin), recuperaremos a sus amigos, los marcaremos como recuperados, y para cada uno de los amigos de steve_coppin cualquiera los agregará al final de la cuenta. base de datos o actualizar su conteo de amigos si ya están en la base de datos.

Dado que los datos del programa están todos almacenados en el disco en una base de datos, la actividad de spidering se puede suspender y reanudar tantas veces como desee sin pérdida de datos.

## Modelado de datos básicos

El verdadero poder de una base de datos relacional es cuando creamos varias tablas y hacemos enlaces entre esas tablas. El acto de decidir cómo dividir los datos de la aplicación en varias tablas y establecer las relaciones entre las tablas se denomina modelado de datos. El documento de diseño que muestra las tablas y sus relaciones se llama modelo de datos.

El modelado de datos es una habilidad relativamente sofisticada y solo presentaremos los conceptos más básicos del modelado de datos relacionales en esta sección. Para obtener más detalles sobre el modelado de datos, puede comenzar con: http://en.wikipedia.org/wiki/Relational_model.

Digamos que para nuestra aplicación de araña de Twitter, en lugar de solo contar los amigos de una persona, queríamos mantener una lista de todas las relaciones entrantes para que pudiéramos encontrar una lista de todos los que están siguiendo una cuenta en particular.

Dado que todos tendrán potencialmente muchas cuentas que los sigan, no podemos simplemente agregar una sola columna a nuestra tabla Twitter. Entonces creamos una nueva tabla que hace un seguimiento de los pares de amigos.

La siguiente es una forma simple de hacer una tabla:

    CREATE TABLE Pals (from_friend TEXT, to_friend TEXT)

Cada vez que nos encontramos con una persona que drchuck está siguiendo, insertaremos una fila del formulario:

    INSERT INTO Pals (from_friend,to_friend) VALUES ('drchuck', 'lhawthorn')

A medida que procesamos los 20 amigos del feed de Twitter drchuck, insertaremos 20 registros con "drchuck" como primer parámetro, por lo que terminaremos duplicando la cadena muchas veces en la base de datos.

Esta duplicación de datos de cadena viola una de las mejores prácticas para la normalización de la base de datos, que básicamente establece que nunca deberíamos poner los mismos datos de cadena en la base de datos más de una vez. Si necesitamos los datos más de una vez, creamos una clave numérica para los datos y hacemos referencia a los datos reales utilizando esta clave.

En términos prácticos, una cadena ocupa mucho más espacio que un entero en el disco y en la memoria de nuestra computadora, y toma más tiempo de procesador para comparar y ordenar. Si solo tenemos unos cientos de entradas, el tiempo de almacenamiento y procesador apenas importa. Pero si tenemos un millón de personas en nuestra base de datos y una posibilidad de 100 millones de enlaces de amigos, es importante poder escanear los datos lo más rápido posible.

Almacenaremos nuestras cuentas de Twitter en una tabla denominada `People` en lugar de la tabla Twitter utilizada en el ejemplo anterior. La tabla `People` tiene una columna adicional para almacenar la clave numérica asociada con la fila para este usuario de Twitter. SQLite tiene una característica que agrega automáticamente el valor clave para cualquier fila que insertemos en una tabla usando un tipo especial de columna de datos `INTEGER PRIMARY KEY`. 

Podemos crear la tabla `People` con esta columna `id` adicional de la siguiente manera:

    CREATE TABLE People
        (id INTEGER PRIMARY KEY, name TEXT UNIQUE, retrieved INTEGER)
    
Tenga en cuenta que ya no mantenemos un conteo de amigos en cada fila de la tabla `People`. Cuando seleccionamos `INTEGER PRIMARY KEY` el tipo de columna `id`, estamos indicando que nos gustaría que SQLite administre esta columna y asigne una clave numérica única a cada fila que insertemos automáticamente. También agregamos la palabra clave `UNIQUE` para indicar que no permitiremos que SQLite inserte dos filas con el mismo valor para `name`.

Ahora, en lugar de crear la tabla anterior `Pals`, creamos una tabla llamada `Follows` con dos columnas de tipo entero `from_id` y `to_id` y una restricción en la tabla que la combinación de `from_id` y `to_id` debe ser único en esta tabla (es decir, no podemos insertar filas duplicadas) en nuestra base de datos.

    CREATE TABLE Follows
        (from_id INTEGER, to_id INTEGER, UNIQUE(from_id, to_id) )

Cuando agregamos cláusulas `UNIQUE` a nuestras tablas, estamos comunicando un conjunto de reglas que estamos pidiendo a la base de datos que aplique cuando intentemos insertar registros. Estamos creando estas reglas como una conveniencia en nuestros programas, como veremos en un momento. Las reglas no nos permiten cometer errores y simplificar la escritura de nuestro código.

En esencia, al crear esta tabla `Follows`, estamos modelando una "relación" donde una persona "sigue" a otra persona y la representa con un par de números que indica que las personas están conectadas y la dirección de la relación.

![Figura 15.4](./figures/15.4.svg)
<center><i>Figura 15.4: Relaciones entre tablas</i></center>

## Programación con múltiples tablas

Ahora vamos a volver a hacer el programa de araña de Twitter utilizando dos tablas, las claves principales y las referencias clave como se describe arriba. Aquí está el código para la nueva versión del programa:

    import urllib.request, urllib.parse, urllib.error
    import twurl
    import json
    import sqlite3
    import ssl

    TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'

    conn = sqlite3.connect('friends.sqlite')
    cur = conn.cursor()

    cur.execute('''CREATE TABLE IF NOT EXISTS People
                (id INTEGER PRIMARY KEY, name TEXT UNIQUE, retrieved INTEGER)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS Follows
                (from_id INTEGER, to_id INTEGER, UNIQUE(from_id, to_id))''')

    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    while True:
        acct = input('Enter a Twitter account, or quit: ')
        if (acct == 'quit'): break
        if (len(acct) < 1):
            cur.execute('SELECT id, name FROM People WHERE retrieved=0 LIMIT 1')
            try:
                (id, acct) = cur.fetchone()
            except:
                print('No unretrieved Twitter accounts found')
                continue
        else:
            cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
                        (acct, ))
            try:
                id = cur.fetchone()[0]
            except:
                cur.execute('''INSERT OR IGNORE INTO People
                            (name, retrieved) VALUES (?, 0)''', (acct, ))
                conn.commit()
                if cur.rowcount != 1:
                    print('Error inserting account:', acct)
                    continue
                id = cur.lastrowid

        url = twurl.augment(TWITTER_URL, {'screen_name': acct, 'count': '100'})
        print('Retrieving account', acct)
        try:
            connection = urllib.request.urlopen(url, context=ctx)
        except Exception as err:
            print('Failed to Retrieve', err)
            break

        data = connection.read().decode()
        headers = dict(connection.getheaders())

        print('Remaining', headers['x-rate-limit-remaining'])

        try:
            js = json.loads(data)
        except:
            print('Unable to parse json')
            print(data)
            break

        # Debugging
        # print(json.dumps(js, indent=4))

        if 'users' not in js:
            print('Incorrect JSON received')
            print(json.dumps(js, indent=4))
            continue

        cur.execute('UPDATE People SET retrieved=1 WHERE name = ?', (acct, ))

        countnew = 0
        countold = 0
        for u in js['users']:
            friend = u['screen_name']
            print(friend)
            cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
                        (friend, ))
            try:
                friend_id = cur.fetchone()[0]
                countold = countold + 1
            except:
                cur.execute('''INSERT OR IGNORE INTO People (name, retrieved)
                            VALUES (?, 0)''', (friend, ))
                conn.commit()
                if cur.rowcount != 1:
                    print('Error inserting account:', friend)
                    continue
                friend_id = cur.lastrowid
                countnew = countnew + 1
            cur.execute('''INSERT OR IGNORE INTO Follows (from_id, to_id)
                        VALUES (?, ?)''', (id, friend_id))
        print('New accounts=', countnew, ' revisited=', countold)
        print('Remaining', headers['x-rate-limit-remaining'])
        conn.commit()
    cur.close()

Este programa se está empezando a complicar un poco, pero ilustra los patrones que debemos usar cuando utilizamos claves enteras para vincular tablas. Los patrones básicos son:
1. Crear tablas con claves principales y restricciones.
2. Cuando tenemos una clave lógica para una persona (es decir, el nombre de la cuenta) y necesitamos el valor id para la persona, dependiendo de si la persona ya está en la tabla People o no, debemos: (1) buscar a la persona en la tabla `People` y recuperar el valor `id` para la persona o (2) agregar la persona a la tabla `People` y obtener el valor `id` de la fila recién agregada.
3. Inserta la fila que captura la relación "sigue".

Cubriremos cada uno de estos a su vez.

### Restricciones en las tablas de la base de datos

A medida que diseñamos las estructuras de nuestra tabla, podemos decirle al sistema de la base de datos que nos gustaría aplicar algunas reglas sobre nosotros. Estas reglas nos ayudan a no cometer errores e introducir datos incorrectos en las tablas. Cuando creamos nuestras tablas:

    cur.execute('''CREATE TABLE IF NOT EXISTS People
     (id INTEGER PRIMARY KEY, name TEXT UNIQUE, retrieved INTEGER)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS Follows
     (from_id INTEGER, to_id INTEGER, UNIQUE(from_id, to_id))''')

Indicamos que la columna `name` en la tabla `People` debe ser `UNIQUE`. También indicamos que la combinación de los dos números en cada fila de la tabla `Follows` debe ser única. Estas restricciones nos impiden cometer errores, como agregar la misma relación más de una vez.

Podemos aprovechar estas restricciones en el siguiente código:

    cur.execute('''INSERT OR IGNORE INTO People (name, retrieved)
     VALUES ( ?, 0)''', ( friend, ) )
 
Agregamos la cláusula `OR IGNORE` a nuestra declaración `INSERT` para indicar que si este particular `INSERT` causaría una violación de la regla `name` "debe ser único", el sistema de la base de datos puede ignorar el `INSERT`. Estamos utilizando la restricción de la base de datos como una red de seguridad para asegurarnos de que no hacemos algo incorrecto inadvertidamente.

Del mismo modo, el siguiente código asegura que no agreguemos exactamente la misma relación `Follow` dos veces.

    cur.execute('''INSERT OR IGNORE INTO Follows
        (from_id, to_id) VALUES (?, ?)''', (id, friend_id) )
        
Nuevamente, simplemente le pedimos a la base de datos que ignore nuestro intento `INSERT` si viola la restricción de exclusividad que hemos especificado para las filas `Follows`.

### Recuperar y / o insertar un registro

Cuando solicitamos al usuario una cuenta de Twitter, si la cuenta existe, debemos buscar su valor `id`. Si la cuenta aún no existe en la tabla `People`, debemos insertar el registro y obtener el valor `id` de la fila insertada.

Este es un patrón muy común y se realiza dos veces en el programa anterior. Este código muestra cómo buscamos la cuenta `id` de un amigo cuando hemos extraído un `screen_name` desde un nodo `user` en el JSON de Twitter recuperado.

Dado que con el tiempo será cada vez más probable que la cuenta ya esté en la base de datos, primero verificamos si el registro `People` existe utilizando una declaración `SELECT`.

Si todo va bien dentro de la sección `try`, recuperamos el registro usando |fetchone()| y luego recuperamos el primer (y único) elemento de la tupla devuelta y lo almacenamos `friend_id`.

Si `SELECT` falla, el código `fetchone()[0]` fallará y el control se transferirá a la sección `except`.

        friend = u['screen_name']
        cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
            (friend, ) )
        try:
            friend_id = cur.fetchone()[0]
            countold = countold + 1
        except:
            cur.execute('''INSERT OR IGNORE INTO People (name, retrieved)
                VALUES ( ?, 0)''', ( friend, ) )
            conn.commit()
            if cur.rowcount != 1 :
                print('Error inserting account:',friend)
                continue
            friend_id = cur.lastrowid
            countnew = countnew + 1

Si terminamos en el código `except`, simplemente significa que no se encontró la fila, por lo que debemos insertar la fila. Usamos `INSERT OR IGNOR`E solo para evitar errores y luego llamamos `commit()` para forzar que la base de datos realmente se actualice. Después de que la escritura esté lista, podemos verificar la cantidad `cur.rowcount` de filas afectadas. Dado que estamos intentando insertar una sola fila, si el número de filas afectadas es algo distinto de 1, es un error.

Si `INSERT` tiene éxito, podemos ver `cur.lastrowid` y que valor asigna la base de datos a la columna `id` en nuestra fila recién creada.

### Almacenar la relación de amistad

Una vez que conocemos el valor clave tanto para el usuario de Twitter como para el amigo en el JSON, es una cuestión simple insertar los dos números en la tabla `Follows` con el siguiente código:

    cur.execute('INSERT OR IGNORE INTO Follows (from_id, to_id) VALUES (?, ?)',
        (id, friend_id) )

Tenga en cuenta que permitimos que la base de datos se ocupe de evitar que "insertemos dos veces" una relación creando la tabla con una restricción de exclusividad y luego agregando `OR IGNORE` a nuestra declaración `INSERT`.

Aquí hay una muestra de ejecución de este programa:

    Enter a Twitter account, or quit:
    No unretrieved Twitter accounts found
    Enter a Twitter account, or quit: drchuck
    Retrieving http://api.twitter.com/1.1/friends ...
    New accounts= 20  revisited= 0
    Enter a Twitter account, or quit:
    Retrieving http://api.twitter.com/1.1/friends ...
    New accounts= 17  revisited= 3
    Enter a Twitter account, or quit:
    Retrieving http://api.twitter.com/1.1/friends ...
    New accounts= 17  revisited= 3
    Enter a Twitter account, or quit: quit

Comenzamos con la cuenta drchuck y luego permitimos que el programa seleccione automáticamente las siguientes dos cuentas para recuperar y agregar a nuestra base de datos.

A continuación, se muestran las primeras filas en las tablas `People` y `Follows` después de completar esta ejecución:

    People:
    (1, 'drchuck', 1)
    (2, 'opencontent', 1)
    (3, 'lhawthorn', 1)
    (4, 'steve_coppin', 0)
    (5, 'davidkocher', 0)
    55 rows.
    Follows:
    (1, 2)
    (1, 3)
    (1, 4)
    (1, 5)
    (1, 6)
    60 rows.

Se puede ver los campos `id`, `name` y `visited` de la tabla `People` y ver los números de ambos extremos de la relación en la tabla `Follows`. En la tabla `People`, podemos ver que las tres primeras personas han sido visitadas y que se han recuperado sus datos. Los datos en la tabla `Follows` indican que drchuck (usuario 1) es un amigo para todas las personas que se muestran en las primeras cinco filas. Esto tiene sentido porque los primeros datos que recuperamos y almacenamos fueron los amigos de Twitter de drchuck. Si `Follows` tuviera que imprimir más filas de la tabla, también vería a los amigos de los usuarios 2 y 3.

## Tres tipos de llaves

Ahora que hemos empezado a construir un modelo de datos que coloque nuestros datos en varias tablas vinculadas y que vincule las filas en esas tablas utilizando claves, debemos consultar algunos términos relacionados con las claves. En general, hay tres tipos de claves utilizadas en un modelo de base de datos.
* Una clave lógica es una clave que el "mundo real" podría usar para buscar una fila. En nuestro modelo de datos de ejemplo, el campo `name` es una clave lógica. Es el nombre de pantalla del usuario y, de hecho, buscamos la fila de un usuario varias veces en el programa usando el campo name. A menudo encontrará que tiene sentido agregar una restricción `UNIQUE` a una clave lógica. Como la clave lógica es cómo buscamos una fila del mundo exterior, tiene poco sentido permitir varias filas con el mismo valor en la tabla.
* Una clave principal suele ser un número que la base de datos asigna automáticamente. Por lo general, no tiene ningún significado fuera del programa y solo se usa para vincular filas de diferentes tablas. Cuando queremos buscar una fila en una tabla, generalmente buscar la fila usando la clave principal es la forma más rápida de encontrar la fila. Dado que las claves primarias son números enteros, ocupan muy poco espacio de almacenamiento y se pueden comparar u ordenar rápidamente. En nuestro modelo de datos, el campo `id` es un ejemplo de una clave principal.
* Una clave externa suele ser un número que apunta a la clave principal de una fila asociada en una tabla diferente. Un ejemplo de una clave externa en nuestro modelo de datos es el `from_id`.

Estamos utilizando una convención de nomenclatura de llamar siempre el nombre del campo de clave principal `id` y anexar el sufijo `_id` a cualquier nombre de campo que sea una clave externa.

## Usando JOIN para recuperar datos

Ahora que hemos seguido las reglas de normalización de la base de datos y hemos separado los datos en dos tablas, vinculadas entre sí mediante claves primarias y externas, debemos ser capaces de crear una `SELECT` que reensamble los datos en todas las tablas.

SQL usa la cláusula `JOIN` para volver a conectar estas tablas. En la cláusula `JOIN`, especifique los campos que se utilizan para volver a conectar las filas entre las tablas.

El siguiente es un ejemplo de `SELECT` con una cláusula `JOIN`:

    SELECT * FROM Follows JOIN People
        ON Follows.from_id = People.id WHERE People.id = 1

La cláusula `JOIN` indica que los campos que están seleccionando cruzan las tablas `Follows` y `People`. La cláusula `ON` indica cómo las dos tablas se van a unir: Tome las filas de `Follows` y añadir la fila de `People` los que el campo `from_id` en `Follows` es el mismo el valor `id` en la tabla `People`.

![Figura 15.5](./figures/15.5.svg)
<center><i>Figura 15.5: Conexión de tablas con JOIN.</i></center>

El resultado de `JOIN` es crear "metarows" extralargos que tienen tanto los campos `People` como los de coincidencia `Follows`. Cuando hay más de una coincidencia entre el campo `id` desde `People` y `from_id` desde `People`, entonces `JOIN` crea un metarow para cada uno de los pares de filas coincidentes, duplicando datos según sea necesario. El siguiente código demuestra los datos que tendremos en la base de datos después de que el programa araña de Twitter de varias tablas (arriba) se haya ejecutado varias veces.

    import sqlite3

    conn = sqlite3.connect('friends.sqlite')
    cur = conn.cursor()

    cur.execute('SELECT * FROM People')
    count = 0
    print('People:')
    for row in cur:
        if count < 5: print(row)
        count = count + 1
    print(count, 'rows.')

    cur.execute('SELECT * FROM Follows')
    count = 0
    print('Follows:')
    for row in cur:
        if count < 5: print(row)
        count = count + 1
    print(count, 'rows.')

    cur.execute('''SELECT * FROM Follows JOIN People
                ON Follows.to_id = People.id
                WHERE Follows.from_id = 2''')
    count = 0
    print('Connections for id=2:')
    for row in cur:
        if count < 5: print(row)
        count = count + 1
    print(count, 'rows.')

    cur.close()

En este programa, primero volcamos `People`, `Follows` y luego volcamos un subconjunto de los datos en las tablas unidas.

Aquí está la salida del programa:

    python twjoin.py
    People:
    (1, 'drchuck', 1)
    (2, 'opencontent', 1)
    (3, 'lhawthorn', 1)
    (4, 'steve_coppin', 0)
    (5, 'davidkocher', 0)
    55 rows.
    Follows:
    (1, 2)
    (1, 3)
    (1, 4)
    (1, 5)
    (1, 6)
    60 rows.
    Connections for id=2:
    (2, 1, 1, 'drchuck', 1)
    (2, 28, 28, 'cnxorg', 0)
    (2, 30, 30, 'kthanos', 0)
    (2, 102, 102, 'SomethingGirl', 0)
    (2, 103, 103, 'ja_Pac', 0)
    20 rows.

Usted ve las columnas de `People` y las tablas `Follows` y el último conjunto de filas es el resultado de `SELECT` la cláusula con `JOIN`.

En la última selección, estamos buscando cuentas que sean amigos de "opencontent" (es decir, `People`.`id=2`).

En cada uno de los "metadatos" en la última selección, las dos primeras columnas son de la tabla `Follows`, seguidas de las columnas tres a cinco de la tabla `People`. También puede ver que la segunda columna `Follows.to_id` coincide con la tercera columna `People.id` en cada uno de los "metadatos" unidos.

## Resumen

Este capítulo ha cubierto una gran cantidad de terreno para darle una visión general de los conceptos básicos del uso de una base de datos en Python. Es más complicado escribir el código para usar una base de datos para almacenar datos que los diccionarios o archivos planos de Python, por lo que hay pocas razones para usar una base de datos a menos que su aplicación realmente necesite las capacidades de una base de datos. Las situaciones donde una base de datos puede ser bastante útil son: (1) cuando su aplicación necesita hacer pequeñas actualizaciones aleatorias dentro de un gran conjunto de datos, (2) cuando sus datos son tan grandes que no caben en un diccionario y necesita buscar la información repetidamente, o (3) cuando tiene un proceso de ejecución prolongada que desea que pueda detener y reiniciar y retener los datos de una ejecución a la siguiente.

Puede construir una base de datos simple con una sola tabla para satisfacer muchas necesidades de la aplicación, pero la mayoría de los problemas requerirán varias tablas y enlaces/relaciones entre filas en tablas diferentes. Cuando comienzas a hacer enlaces entre tablas, es importante hacer un diseño cuidadoso y seguir las reglas de normalización de la base de datos para hacer el mejor uso de las capacidades de la base de datos. Dado que la principal motivación para usar una base de datos es que tiene que lidiar con una gran cantidad de datos, es importante modelar sus datos de manera eficiente para que sus programas se ejecuten lo más rápido posible.

## Depuración

Un patrón común cuando se está desarrollando un programa de Python para conectarse a una base de datos SQLite será ejecutar un programa de Python y verificar los resultados usando el navegador de la base de datos para SQLite. El navegador le permite verificar rápidamente si su programa funciona correctamente.

Debe tener cuidado porque SQLite se preocupa de evitar que dos programas cambien los mismos datos al mismo tiempo. Por ejemplo, si abre una base de datos en el navegador y realiza un cambio en la base de datos y aún no ha presionado el botón "guardar" en el navegador, el navegador "bloquea" el archivo de base de datos y evita que otros programas accedan al archivo. En particular, su programa Python no podrá acceder al archivo si está bloqueado.

Entonces, una solución es asegurarse de cerrar el navegador de la base de datos o usar el menú Archivo para cerrar la base de datos en el navegador antes de intentar acceder a la base de datos desde Python para evitar el problema de falla del código de Python porque la base de datos está bloqueada.

## Glosario

* **atributo:** Uno de los valores dentro de una tupla. Más comúnmente llamado "columna" o "campo".
* **restricción:** Cuando le decimos a la base de datos que aplique una regla en un campo o una fila en una tabla. Una limitación común es insistir en que no puede haber valores duplicados en un campo particular (es decir, todos los valores deben ser únicos).
* **cursor:** Un cursor le permite ejecutar comandos SQL en una base de datos y recuperar datos de la base de datos. Un cursor es similar a un socket o identificador de archivo para conexiones de red y archivos, respectivamente.
* **navegador de base:** Una pieza de software que le permite conectarse directamente a una base de datos y manipular la base de datos directamente sin escribir un programa.
* **clave externa:** Una tecla numérica que apunta a la clave principal de una fila en otra tabla. Las claves foráneas establecen relaciones entre filas almacenadas en diferentes tablas.
* **índice:** Datos adicionales que el software de la base de datos mantiene como filas e inserta en una tabla para realizar búsquedas muy rápido.
* **llave lógica:** Una clave que el "mundo exterior" usa para buscar una fila en particular. Por ejemplo, en una tabla de cuentas de usuario, la dirección de correo electrónico de una persona podría ser un buen candidato como la clave lógica para los datos del usuario.
* **normalización:** Diseñando un modelo de datos para que no se repliquen los datos. Almacenamos cada elemento de datos en un lugar de la base de datos y lo referenciamos en otro lugar utilizando una clave externa.
* **clave primaria:** Una tecla numérica asignada a cada fila que se usa para referirse a una fila en una tabla de otra tabla. A menudo, la base de datos está configurada para asignar automáticamente claves primarias a medida que se insertan las filas.
* **relación:** Un área dentro de una base de datos que contiene tuplas y atributos. Más típicamente llamado una tabla.
* **tupla:** Una sola entrada en una tabla de base de datos que es un conjunto de atributos. Más típicamente llamado "fila".

<!--NAVIGATION-->
| [Indice](indice.ipynb) | 

< [Capítulo 14 - Programación orientada a objetos](cap14.ipynb) | [Capítulo 16 - Visualización de datos](cap16.ipynb) >